# Synaptomes Notebook

In this notebook, we will attempt to cluster large synapse data using the FlashX Gaussian Mixture Model (GMM) code.

# Data

In this experiment, we will utilize 20 channel array tomagraphy data.

The process of collecting array tomography data is relatively straightforward. Brain tissue is embedded with plastic, and this plastic-embedded tissue is then sectioned into ultrathin 2D chunks which can then be imaged using various wavelengths of light with a microscope. After imaging, the 2D sections can be reconstructed to yield a representation of the original 3D object.

Each protein has a unique "protein fluorescence" which gives it a particular color under different light intensities. In this data, we look at 20 different channels of light intensities, corresponding to the fluorescences of 20 proteins we want to look at. We cut out a 10x10x10 voxel cube around putative synapse locations, and for each channel sum the intensities of all 10x10x10 voxels to form a 20-dimensional feature vector for each putative synapse location. 

# Gaussian Mixture Model

In this setting, we assume that the different synapse types can be expressed as a multivariate Gaussian with parameters $\theta = \{\mu, \Sigma\}$, with one dimension for each channel. We will use the unsupervised Gaussian Mixture Model technique to find the "most-likely" parameters for our gaussians with arbitrary numbers of classes.

For each data point, we know that there exists some unknown, latent variable $z$ that represents the true synapse class of the data point. Under this setting, the probability that a given feature vector $x$ can be explained by some arbitrary synapse type $c$ as:

\begin{align*}
    p(x | z = c) = \mathcal{N}(x | \mu_c, \Sigma_c)
\end{align*}

Then the probability of our feature vector being explained by any synapse type can be defined:

\begin{align*}
    p(x) &= \sum_c p(x | z = c) \\
    &= \sum_c \pi_c \mathcal{N}(x | \mu_c, \Sigma_c)
\end{align*}

where $\pi_c = p(z = c)$, or the total probability that our latent $z$ is the class $c$. 


In [1]:
library(rhdf5)
library(FlashRLearn)

Loading required package: pcg
Loading required package: FlashR
Loading required package: RSpectra
Loading required package: Rcpp

Attaching package: 'FlashR'

The following objects are masked from 'package:base':

    cbind, pmax, pmin, rbind



In [2]:
data <- h5read("/data/k15f0.h5", "datatable/F0")
fm.data <- fm.as.matrix(as.matrix(data))

In [3]:
source("/FlashX/FlashR-learn/R/GMM.R")
source('./analyze_performance.R')

In [17]:
ks <- seq(from=2, to=20, by=2)
scores <- matrix(0, nrow=length(ks), 2)
results.bic <- array(NaN, dim=c(length(ks)))
results.time <- array(NaN, dim=c(length(ks)))
results.mem <- array(NaN, dim=c(length(ks)))
results.means <- list()

# FlashX implementation
for (i in 1:length(ks)) {
    res <- analyze_performance(GMM.fit, list(fm.data, k=ks[i], cov.type = "diag"))
    results.bic[i] <- BIC(res$out)
    results.time[i] <- res$time
    results.mem[i] <- res$mem
    results.means[[i]] <- res$out$means
}

In [18]:
print(res$out$loglik)
print(max(results.bic))

FlashR matrix : 1119299 rows, 20 columns, is sparse: FALSE
[1] 622813232


In [9]:
for (k in ks) {
    res <- GMM.fit(fm.data, k=k, cov.type = "full")
    scores[k/2, 2] <- BIC(res)
    print(paste("full: k=",k,", BIC=", scores[k/2,2], sep=""))
}

[1] "full: k=2, BIC=615753118.426505"
[1] "full: k=4, BIC=610074783.187261"
[1] "full: k=6, BIC=607935198.354621"
[1] "full: k=8, BIC=606205840.947613"
[1] "full: k=10, BIC=605381567.134337"
[1] "full: k=12, BIC=604526846.922961"
[1] "full: k=14, BIC=604155500.863832"
[1] "full: k=16, BIC=603592808.262421"
[1] "full: k=18, BIC=603287373.245804"
[1] "full: k=20, BIC=602878370.065682"


ERROR: Error in eval(expr, envir, enclos): object 'scores' not found
